In [102]:
# Python code to build Machine Learning model for hurricane intensity forecast  
import pandas as pd # For data manipulation and analysis
pd.set_option('display.max_columns', 500)
import numpy as np # For scientific computing

#!pip3 install sklearn # Install machine learning library
import sklearn # For machine learning library
from sklearn.ensemble import RandomForestClassifier  # Random forest classifier
#from sklearn.ensemble import ExtraTreesClassifier    # Extra tree classifier
from sklearn.metrics import confusion_matrix # Compute confusion matrix to evaluate the accuracy of a classification.
from sklearn.metrics import brier_score_loss  # Compute the Brier score

import matplotlib.pyplot as plt  #plotting library
from global_land_mask import globe # a library to check whether a lat&lon on sea or land


import math

In [81]:
# Gathered online, didn't download the dataset but copied it
# http://www.icatdamageestimator.com/commonsearch?search=2002&x=0&y=0 
# Searched Each Year and Copied the Data into CSV

fname=r'C:\Users\kwonk\Guided Project\RMDS_Hurricane_Forecasting\hurricane_damage_data.csv'

# Read EcoLoss data
Ecoloss = pd.read_csv(r'C:\Users\kwonk\Guided Project\RMDS_Hurricane_Forecasting\data\hurricane_damage_data.csv')

Ecoloss

,STORM NAME,LANDFALL DATE,DAMAGE RANK,CURRENT DAMAGE ($ 2020),BASE DAMAGE ($),LANDFALL STATE,CATEGORY,WINDS(MPH)
0,Great Miami,"Sep 18,1926",1,"242,750,000,000","76,000,000",FL,4,145
1,Galveston,"Sep 08,1900",2,"171,510,000,000","30,000,000",TX,4,140
2,Katrina,"Aug 29,2005",3,"148,240,000,000","125,000,000,000",LA,3,125
3,Harvey,"Aug 25,2017",4,"132,690,000,000","125,000,000,000",TX,4,130
4,Galveston,"Aug 17,1915",5,"121,200,000,000","50,000,000",TX,4,130
...,...,...,...,...,...,...,...,...
243,Florence,"Sep 09,1988",244,"2,000,000","2,920,000",LA,1,80
244,Dean,"Jul 30,1995",245,"2,000,000","500,000",TX,TS,45
245,Floyd,"Oct 12,1987",246,"1,000,000","500,000",FL,1,75
246,Gustav,"Sep 10,2002",247,0,"100,000",NC,TS,65


In [82]:
# Extract month from landfall date
Ecoloss['MONTH'] = Ecoloss['LANDFALL DATE'].apply(lambda x: str(x)[0:3])
Month2Int={'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08','Sep':'09','Oct':'10','Nov':'11','Dec':'12'}
Ecoloss['MONTH']=Ecoloss['MONTH'].apply(lambda x: Month2Int[x])


# Extract date from landfall date
# Ecoloss['DATE'] = Ecoloss['LANDFALL DATE'].apply(lambda x: str(x)[4:6]).astype(int)

# Extract year from landfall date
Ecoloss['YEAR'] = Ecoloss['LANDFALL DATE'].apply(lambda x: str(x)[7:11]).astype(int)

Ecoloss.sort_values("YEAR",inplace=True)

Ecoloss.head()

,STORM NAME,LANDFALL DATE,DAMAGE RANK,CURRENT DAMAGE ($ 2020),BASE DAMAGE ($),LANDFALL STATE,CATEGORY,WINDS(MPH),MONTH,YEAR
1,Galveston,"Sep 08,1900",2,"171,510,000,000","30,000,000",TX,4,140,09,1900
130,Storm 4 in 1901,"Aug 14,1901",131,"830,000,000","1,000,000",LA,1,85,08,1901
247,Storm 3 in 1903,"Sep 13,1903",248,0,0,FL,1,90,09,1903
54,Storm 3 in 1903,"Sep 11,1903",56,"7,410,000,000","670,000",FL,1,90,09,1903
140,Storm 2 in 1904,"Sep 14,1904",141,"640,000,000","1,000,000",SC,1,80,09,1904


In [83]:
## Selecting Relevant Years in case year search serached for other column values

Ecoloss=Ecoloss[(Ecoloss['YEAR']>=1982) & (Ecoloss['YEAR']<=2017)]

In [84]:
Ecoloss['STORM NAME'].iloc[1][0:4]

'Chri'

In [85]:
# set a 'key' to match ships dataset
# upper Name[0:4]+Year+Month(int)
Ecoloss['KEY']='NA'
Ecoloss=Ecoloss.reset_index(drop=True)

for i in range(0,Ecoloss.shape[0]):
    
    Ecoloss.loc[i,'KEY']='-'.join([Ecoloss['STORM NAME'].iloc[i][0:4].upper(),Ecoloss['YEAR'].iloc[i].astype(str)])

Ecoloss.head()

Ecoloss.drop_duplicates(subset=['KEY'])

C:\Users\kwonk\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,STORM NAME,LANDFALL DATE,DAMAGE RANK,CURRENT DAMAGE ($ 2020),BASE DAMAGE ($),LANDFALL STATE,CATEGORY,WINDS(MPH),MONTH,YEAR,KEY
0,Subtrop 1 1982,"Jun 18,1982",204,"84,000,000","10,000,000",FL,TS,45,06,1982,SUBT-1982
1,Chris,"Sep 11,1982",242,"3,000,000","2,000,000",LA,TS,65,09,1982,CHRI-1982
2,Alicia,"Aug 18,1983",26,"22,330,000,000","3,000,000,000",TX,3,115,08,1983,ALIC-1983
3,Isidore,"Sep 27,1984",235,"8,000,000","1,000,000",FL,TS,60,09,1984,ISID-1984
4,Diana,"Sep 13,1984",144,"620,000,000","65,000,000",NC,2,110,09,1984,DIAN-1984
...,...,...,...,...,...,...,...,...,...,...,...
97,Hermine,"Sep 02,2016",146,"610,000,000","550,000,000",FL,1,80,09,2016,HERM-2016
98,Matthew,"Oct 08,2016",44,"11,260,000,000","10,000,000,000",SC,2,100,10,2016,MATT-2016
99,Harvey,"Aug 25,2017",4,"132,690,000,000","125,000,000,000",TX,4,130,08,2017,HARV-2017
100,Nate,"Oct 08,2017",176,"230,000,000","225,000,000",MS,1,75,10,2017,NATE-2017


##### 1.2 Ships data

In [86]:
#====================================
#Read SHIPS spread-sheet data
#====================================
# Set up the location of the SHIPS data

## Pre-cleaned SHIPS Dataset

fname='https://raw.githubusercontent.com/akshaym2497/RMDS_Hurricane_Forecasting/master/data/SHIPS_RII_fcst_ATL_Alicia_from1982%20_Wanying.csv'
#fname='Dataset_SHIPS_RII_EPAC.csv'

# Read SHIPS data
ships = pd.read_csv(fname)
ships

,NAME,DATE,HOUR,VMAX,LAT,LON,MSLP,ID,DELV12,DELV24,DELV36,DELV48,PER,SHRD,D200,RHLO,PX30,SDBT,POT,RHCN,TPW,PC2,SHRD2,SHRG,DIVC,U200,EPSS,ENSS,TPWC,RSST
0,ALBE,820602,12,20,21.7,-87.1,1005,AL011982,10,30,45,25,9999,26.4,64,70,9999,9999.0,-103,0,113,9999,25.1,29.6,49,22.4,5.1,1.6,61.8,28.0
1,ALBE,820602,18,25,22.2,-86.5,1004,AL011982,15,50,30,15,9999,19.4,65,68,9999,9999.0,-97,0,178,9999,21.6,30.2,45,18.7,4.8,2.0,61.9,28.0
2,ALBE,820603,0,30,22.6,-85.8,1003,AL011982,20,35,15,0,10,19.3,84,70,9999,9999.0,-87,0,116,9999,18.6,29.8,75,16.9,5.2,1.8,61.0,27.8
3,ALBE,820603,6,40,22.8,-85.0,1001,AL011982,35,15,0,-15,15,17.7,70,70,9999,9999.0,-75,0,439,9999,22.1,36.2,88,17.9,4.8,1.8,60.3,27.6
4,ALBE,820603,12,50,23.2,-84.2,995,AL011982,15,-5,-20,-25,20,17.4,51,66,9999,9999.0,-71,0,758,9999,24.3,34.0,60,20.9,2.2,2.2,61.5,27.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11777,RINA,171108,0,45,34.6,-48.7,999,AL192017,0,0,9999,9999,5,20.5,29,71,31,24.3,-37,0,1000,123,25.2,33.1,36,22.0,1.3,5.0,42.9,23.5
11778,RINA,171108,6,50,36.4,-48.7,996,AL192017,-5,9999,9999,9999,10,20.8,19,73,15,14.7,-28,0,969,90,24.5,32.3,16,21.8,1.1,5.4,43.0,22.9
11779,RINA,171108,12,45,38.3,-48.8,994,AL192017,0,9999,9999,9999,0,21.2,12,74,9,13.5,-29,0,966,126,25.2,33.8,3,23.0,0.0,6.7,43.0,22.7
11780,RINA,171108,18,45,40.1,-49.0,992,AL192017,9999,9999,9999,9999,-5,22.5,14,70,7,11.5,-14,0,948,75,29.1,38.9,20,28.4,0.0,9.8,44.3,21.3


In [87]:
ships = ships.replace(9999,np.NaN)

# drop NaNs
ships=ships.dropna()

# Pad the date columns with 00 for the year 2000
ships['DATE'] = ships['DATE'].apply(lambda x: str(x).zfill(6))

# Extract month from date
ships['MONTH'] = ships['DATE'].apply(lambda x: str(x)[2:4])

# Extract year from date
ships['YEAR'] = ships['DATE'].apply(lambda x: ('19' + str(x)[0:2]) if (str(x)[0:1]!= '0' and str(x)[0:1]!= '1') else ('20' + str(x)[0:2]))


In [88]:
ships

,NAME,DATE,HOUR,VMAX,LAT,LON,MSLP,ID,DELV12,DELV24,DELV36,DELV48,PER,SHRD,D200,RHLO,PX30,SDBT,POT,RHCN,TPW,PC2,SHRD2,SHRG,DIVC,U200,EPSS,ENSS,TPWC,RSST,MONTH,YEAR
72,DEBB,820914,0,30,21.5,-71.0,1009.0,AL061982,5.0,35.0,45.0,65.0,0.0,16.9,26,65,49.0,28.6,-110,0,126,35.0,17.0,20.8,30.0,5.6,5.4,2.4,58.5,28.8,09,1982
73,DEBB,820914,6,30,22.4,-71.8,1008.0,AL061982,25.0,35.0,60.0,65.0,0.0,20.7,23,67,47.0,28.4,-112,0,591,23.0,18.8,23.5,18.0,8.7,4.6,2.5,58.3,28.7,09,1982
74,DEBB,820914,12,35,23.5,-71.9,1006.0,AL061982,30.0,40.0,60.0,60.0,5.0,8.7,40,66,49.0,30.0,-110,0,640,31.0,14.2,18.1,64.0,7.2,4.5,2.5,57.2,28.8,09,1982
75,DEBB,820914,18,55,24.6,-71.8,1000.0,AL061982,10.0,35.0,40.0,40.0,25.0,13.3,39,64,74.0,19.6,-82,0,166,-60.0,11.2,16.9,46.0,8.2,5.9,2.6,58.6,28.8,09,1982
76,DEBB,820915,0,65,25.6,-71.0,993.0,AL061982,10.0,30.0,30.0,30.0,30.0,17.9,53,62,87.0,10.7,-74,0,479,-83.0,16.2,24.4,71.0,13.0,6.0,2.5,59.3,28.9,09,1982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11756,OPHE,171013,18,80,32.0,-32.5,973.0,AL172017,15.0,20.0,10.0,0.0,-10.0,9.7,5,62,45.0,12.2,-24,0,580,24.0,14.3,17.0,18.0,24.0,0.2,4.1,52.5,26.0,10,2017
11770,RINA,171106,6,30,29.0,-50.7,1010.0,AL192017,0.0,10.0,10.0,20.0,0.0,14.8,14,67,33.0,29.4,-94,0,864,118.0,24.0,30.0,9.0,18.0,1.6,4.0,45.7,26.2,11,2017
11771,RINA,171106,12,30,29.1,-50.4,1010.0,AL192017,5.0,10.0,15.0,15.0,0.0,14.3,5,66,28.0,26.2,-93,0,934,126.0,23.4,29.9,3.0,16.2,1.0,4.1,46.6,26.1,11,2017
11772,RINA,171106,18,30,29.4,-50.2,1009.0,AL192017,10.0,10.0,20.0,15.0,0.0,18.8,3,65,27.0,24.5,-90,0,895,138.0,24.6,30.7,8.0,18.0,1.1,4.1,46.0,25.9,11,2017


In [89]:
ships=ships.reset_index(drop=True)
# set a 'key' to match ships dataset
# upper Name[0:4]+Year+Month(int)
ships['KEY']=0
for i in range(0,ships.shape[0]):
    ships.loc[i,'KEY']='-'.join([ships.loc[i,'NAME'],ships.loc[i,'YEAR']])
ships.head()

,NAME,DATE,HOUR,VMAX,LAT,LON,MSLP,ID,DELV12,DELV24,DELV36,DELV48,PER,SHRD,D200,RHLO,PX30,SDBT,POT,RHCN,TPW,PC2,SHRD2,SHRG,DIVC,U200,EPSS,ENSS,TPWC,RSST,MONTH,YEAR,KEY
0,DEBB,820914,0,30,21.5,-71.0,1009.0,AL061982,5.0,35.0,45.0,65.0,0.0,16.9,26,65,49.0,28.6,-110,0,126,35.0,17.0,20.8,30.0,5.6,5.4,2.4,58.5,28.8,09,1982,DEBB-1982
1,DEBB,820914,6,30,22.4,-71.8,1008.0,AL061982,25.0,35.0,60.0,65.0,0.0,20.7,23,67,47.0,28.4,-112,0,591,23.0,18.8,23.5,18.0,8.7,4.6,2.5,58.3,28.7,09,1982,DEBB-1982
2,DEBB,820914,12,35,23.5,-71.9,1006.0,AL061982,30.0,40.0,60.0,60.0,5.0,8.7,40,66,49.0,30.0,-110,0,640,31.0,14.2,18.1,64.0,7.2,4.5,2.5,57.2,28.8,09,1982,DEBB-1982
3,DEBB,820914,18,55,24.6,-71.8,1000.0,AL061982,10.0,35.0,40.0,40.0,25.0,13.3,39,64,74.0,19.6,-82,0,166,-60.0,11.2,16.9,46.0,8.2,5.9,2.6,58.6,28.8,09,1982,DEBB-1982
4,DEBB,820915,0,65,25.6,-71.0,993.0,AL061982,10.0,30.0,30.0,30.0,30.0,17.9,53,62,87.0,10.7,-74,0,479,-83.0,16.2,24.4,71.0,13.0,6.0,2.5,59.3,28.9,09,1982,DEBB-1982


In [94]:
Merge = pd.merge(ships, Ecoloss, on=['KEY'])


Merge['is_land']=globe.is_land(Merge['LAT'],Merge['LON'])

Merge=Merge[['KEY','DATE','HOUR','VMAX','LAT','LON','MONTH_x','YEAR_x','CURRENT DAMAGE ($ 2020)','BASE DAMAGE ($)','is_land','DELV24','PER','SHRD','D200','RHLO','PX30','SDBT','POT','RHCN','TPW'
             ,'PC2','SHRD2','SHRG','DIVC','U200','EPSS','ENSS','TPWC','RSST']]




In [96]:
## LATLONG GDP Data FROM Yale

# use grid econ data created by Yale University
fname='https://raw.githubusercontent.com/akshaym2497/RMDS_Hurricane_Forecasting/master/data/USA_econData_latLonLevel_Wanying.csv'

# Read EcoLoss data
gridGDP = pd.read_csv(fname)
gridGDP.sort_values('lat').head(60)

,lat,longitude,cellid,country,newcountryid,popgpw_1990_27_rswb,popgpw_1995_27_rswb,popgpw_2000_27_rswb,popgpw_2005_211_rswb,GDPMER90,GDPMER95,GDPMER00,GDPMER05,GDPPPP90,GDPPPP95,GDPPPP00,GDPPPP05
0,18,-156,17844,United States,840,1.796783e+02,2.061869e+02,2.314562e+02,2.492896e+02,0.004709,0.005004,0.005896,0.007603,0.004709,0.005004,0.005896,0.007603
1,19,-157,18843,United States,840,1.981229e+03,2.424067e+03,2.902888e+03,3.126819e+03,0.051923,0.058826,0.073942,0.095359,0.051923,0.058826,0.073942,0.095359
2,19,-156,18844,United States,840,1.002581e+05,1.109609e+05,1.209425e+05,1.302614e+05,2.627496,2.692736,3.080648,3.972616,2.627496,2.692736,3.080648,3.972616
3,19,-155,18845,United States,840,6.712525e+03,8.332463e+03,1.017833e+04,1.096183e+04,0.175917,0.202207,0.259263,0.334306,0.175917,0.202207,0.259263,0.334306
4,20,-158,19842,United States,840,2.682194e+02,3.119679e+02,3.547890e+02,3.821110e+02,0.008220,0.009034,0.011086,0.016203,0.008220,0.009034,0.011086,0.016203
5,20,-157,19843,United States,840,9.270767e+04,1.058429e+05,1.190861e+05,1.282588e+05,2.841233,3.064908,3.721094,5.438577,2.841233,3.064908,3.721094,5.438577
6,20,-156,19844,United States,840,1.042983e+04,1.194095e+04,1.344791e+04,1.448409e+04,0.274750,0.291368,0.344655,0.446162,0.274750,0.291368,0.344655,0.446162
7,21,-161,20839,United States,840,2.361131e+02,1.949081e+02,1.577779e+02,1.694111e+02,0.006947,0.005593,0.004519,0.005627,0.006947,0.005593,0.004519,0.005627
8,21,-160,20840,United States,840,2.039054e+04,2.152944e+04,2.228403e+04,2.399974e+04,0.599926,0.617827,0.638196,0.797164,0.599926,0.617827,0.638196,0.797164
9,21,-159,20841,United States,840,1.939011e+05,2.083950e+05,2.292059e+05,2.468665e+05,7.279771,7.442881,8.280501,10.313609,7.279771,7.442881,8.280501,10.313609


In [103]:
# round Lon and Lat in 'Merge4'
Merge['LAT_RoundUP']=Merge['LAT'].apply(lambda x:math.ceil(x))
Merge['LON_RoundUP']=Merge['LON'].apply(lambda x:math.ceil(x))

Merge['LAT_RoundDown']=Merge['LAT'].apply(lambda x:math.floor(x))
Merge['LON_RoundDown']=Merge['LON'].apply(lambda x:math.floor(x))

math.floor(1.1)

1